In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[ 0.0620,  0.0660,  0.0707,  ...,  0.0799,  0.0794,  0.0979],
        [ 0.0418,  0.0394, -0.0640,  ..., -0.0179,  0.0752, -0.0393],
        [-0.0414, -0.0441, -0.0683,  ..., -0.0686,  0.0184,  0.0591],
        ...,
        [-0.0908, -0.0789, -0.0451,  ...,  0.0581, -0.0132,  0.0454],
        [ 0.0086, -0.0148,  0.0012,  ..., -0.0038,  0.0927, -0.0347],
        [-0.0278,  0.0395,  0.0734,  ..., -0.0638,  0.0131,  0.0714]],
       requires_grad=True)
Parameter containing:
tensor([-0.0898, -0.0145,  0.0835, -0.0528, -0.0333,  0.0438,  0.0250, -0.0918,
         0.0284, -0.0444, -0.0172,  0.0484,  0.0569, -0.0043, -0.0932,  0.0785,
         0.0246,  0.0246,  0.0758,  0.03

In [2]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.2020, 0.3365, 0.3266]])


Weight and Bias parameters:
Parameter containing:
tensor([[-0.3348, -0.5433, -0.0779],
        [ 0.1516,  0.1638, -0.4069]], requires_grad=True)
Parameter containing:
tensor([0.0434, 0.0787], requires_grad=True)


Output:
tensor([[-0.2325,  0.0315]], grad_fn=<AddmmBackward0>)


In [3]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.6946, 0.1196, 0.3626, 0.6222, 0.9829, 0.5389],
         [0.8939, 0.2264, 0.8566, 0.4545, 0.2193, 0.0213],
         [0.0516, 0.3496, 0.7357, 0.8895, 0.9897, 0.3665],
         [0.1950, 0.3224, 0.5995, 0.6878, 0.0798, 0.3647],
         [0.4038, 0.8647, 0.7197, 0.3148, 0.8213, 0.8793],
         [0.9650, 0.9923, 0.7416, 0.2851, 0.2516, 0.1265]]])
tensor([[[0.8939, 0.9897],
         [0.9923, 0.8793]]])


In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[10.5452,  7.5498, 22.8696,  8.0273],
         [10.4458, 23.7176,  8.3547, 21.2664],
         [11.4952, 22.5223, 19.6715, 21.1600],
         [16.4992, 21.5518, 17.6668,  9.0798]]])
tensor(15.7764)
tensor([[[-0.2730, -0.7533,  1.7030, -0.6767],
         [-0.8278,  1.1697, -1.1426,  0.8007],
         [-1.6835,  0.8887,  0.2238,  0.5710],
         [ 0.0664,  1.1852,  0.3249, -1.5765]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(8.1956e-08, grad_fn=<MeanBackward0>)


In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0250, 0.4277, 0.0000, 0.0000],
         [0.0000, 0.2480, 0.0000, 0.0000],
         [0.0000, 0.8171, 1.2974, 0.4660]]])
tensor([[[0.0000, 0.0000, 0.0556, 1.2485],
         [0.0250, 0.0000, 0.0000, 0.0000],
         [0.9682, 0.2480, 1.0967, 1.1592],
         [0.2351, 0.8171, 0.0000, 0.0000]]])
